In [1]:
import os.path
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns

from pathlib import Path
from tqdm import tqdm
from time import perf_counter

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
from IPython.display import Markdown, display

In [2]:
base_dir = 'C:/Users/pc/OneDrive/바탕 화면/4-1/도배_하자/'
from glob import glob

In [3]:
train_folder = glob(base_dir + 'train_img증강/*')

train_path = []
for folder in train_folder:
    tmp = glob(folder + '/*')
    train_path += tmp

In [4]:
train_df = pd.DataFrame(train_path, columns=['path'])
train_df['label'] = train_df['path'].apply(lambda x: x.split('\\')[-2])
train_df

,path,label
0,C:/Users/pc/OneDrive/바탕 화면/4-1/도배_하자/train_img...,가구수정
1,C:/Users/pc/OneDrive/바탕 화면/4-1/도배_하자/train_img...,가구수정
2,C:/Users/pc/OneDrive/바탕 화면/4-1/도배_하자/train_img...,가구수정
3,C:/Users/pc/OneDrive/바탕 화면/4-1/도배_하자/train_img...,가구수정
4,C:/Users/pc/OneDrive/바탕 화면/4-1/도배_하자/train_img...,가구수정
...,...,...
15855,C:/Users/pc/OneDrive/바탕 화면/4-1/도배_하자/train_img...,훼손
15856,C:/Users/pc/OneDrive/바탕 화면/4-1/도배_하자/train_img...,훼손
15857,C:/Users/pc/OneDrive/바탕 화면/4-1/도배_하자/train_img...,훼손
15858,C:/Users/pc/OneDrive/바탕 화면/4-1/도배_하자/train_img...,훼손


In [21]:
a=train_df['label']

In [22]:
from collections import Counter


# 각 단어들의 개수 계산
word_counts = Counter(a)

# 결과 출력
for word, count in word_counts.items():
    print(f"{word}: {count} 개")


가구수정: 1023 개
걸레받이수정: 1006 개
곰팡이: 794 개
꼬임: 863 개
녹오염: 671 개
들뜸: 699 개
면불량: 776 개
몰딩수정: 806 개
반점: 742 개
석고수정: 775 개
오염: 904 개
오타공: 790 개
울음: 754 개
이음부불량: 739 개
창틀,문틀수정: 756 개
터짐: 796 개
틈새과다: 791 개
피스: 770 개
훼손: 1405 개


#### 이미지 확인

In [ ]:
import matplotlib.pyplot as plt
 
# 한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)


# 데이터 확인
fig, axes = plt.subplots(nrows=4, ncols=10, figsize=(15, 7),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(train_df.path[i]))
    ax.set_title(train_df.label[i], fontsize = 12)
plt.tight_layout(pad=0.5)
plt.show()

In [ ]:
vc = train_df['label'].value_counts()
plt.figure(figsize=(20,5))
sns.barplot(x = vc.index, y = vc, palette = "rocket")
plt.title("Number of pictures of each category", fontsize = 15)
plt.show()

#### 이미지 불러오기

In [15]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(rescale = 1./255,
                                   validation_split=0.2)



train_gen = train_datagen.flow_from_directory('C:/Users/pc/OneDrive/바탕 화면/4-1/도배_하자/train_img증강',
                                                 target_size = (150, 150),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',subset='training')

val_gen  = train_datagen.flow_from_directory('C:/Users/pc/OneDrive/바탕 화면/4-1/도배_하자/train_img증강',
                                                 target_size = (150, 150),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',subset='validation')

Found 12695 images belonging to 19 classes.
Found 3165 images belonging to 19 classes.


In [7]:
#Initialising the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[150, 150, 3]))

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=19, activation='softmax'))

# Compiling the CNN
cnn.compile(optimizer = 'adam', 
            loss = 'categorical_crossentropy', 
            metrics = ['accuracy'])
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 72, 72, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 41472)             0         
                                                                 
 dense_5 (Dense)             (None, 128)              

In [8]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping()

In [18]:
cnn.fit(x = train_gen, validation_data = val_gen, epochs = 50,callbacks = [early_stopping])

Epoch 1/50
397/397 [==============================] - 312s 787ms/step - loss: 0.3683 - accuracy: 0.8751 - val_loss: 1.0232 - val_accuracy: 0.7526
Epoch 2/50
397/397 [==============================] - 192s 483ms/step - loss: 0.2561 - accuracy: 0.9108 - val_loss: 1.1618 - val_accuracy: 0.7621


#### test

In [19]:
#test
base_dir = 'C:/Users/pc/OneDrive/바탕 화면/4-1/도배_하자/'
test = pd.read_csv(base_dir + 'open/test.csv')
test

,id,img_path
0,TEST_000,./test/000.png
1,TEST_001,./test/001.png
2,TEST_002,./test/002.png
3,TEST_003,./test/003.png
4,TEST_004,./test/004.png
...,...,...
787,TEST_787,./test/787.png
788,TEST_788,./test/788.png
789,TEST_789,./test/789.png
790,TEST_790,./test/790.png


#### 방법0

###### 방법1

In [20]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'C:/Users/pc/OneDrive/바탕 화면/4-1/도배_하자/open',
    classes=['test'],
    target_size=(150,150),
    batch_size=32,
    shuffle=False,
    class_mode='categorical')

Found 792 images belonging to 1 classes.


In [ ]:
# test데이터로 모델 성능 예측
for name, model in cnn.items():
    
    # Predict the label of the test_images
    pred = cnn.predict(test_images)
    #pred = np.argmax(pred,axis=1)

    # Map the label
    labels = (train_gen)
    labels = dict((v,k) for k,v in labels.items())
    pred = [labels[k] for k in pred]

print(pred)

In [21]:
pred = cnn.predict(test_generator)
pred

25/25 [==============================] - 8s 300ms/step


array([[5.6036294e-04, 6.0840892e-03, 2.1205258e-02, ..., 6.6289040e-06,
        4.7879239e-05, 3.6298192e-01],
       [1.7822891e-09, 1.3585093e-06, 4.1861922e-04, ..., 1.9027946e-14,
        1.3184783e-09, 2.1481259e-01],
       [8.1110527e-05, 1.3647151e-07, 2.9893141e-04, ..., 7.4126101e-09,
        8.4786019e-13, 2.4308085e-03],
       ...,
       [1.5144444e-07, 1.3366954e-03, 6.5987590e-03, ..., 2.6755252e-08,
        4.9487923e-08, 1.7761309e-03],
       [5.8958805e-05, 2.1995723e-04, 1.2347322e-03, ..., 1.4337800e-07,
        1.2776523e-04, 3.4907728e-01],
       [2.7138801e-06, 4.4960600e-09, 1.2336935e-07, ..., 4.2637029e-11,
        2.0275574e-13, 1.0130424e-08]], dtype=float32)

In [27]:
result = [np.argmax(value) for value in pred]   # 예측 값중 가장 높은 클래스 반환
result

[10,
 10,
 3,
 6,
 10,
 17,
 10,
 10,
 3,
 11,
 1,
 6,
 10,
 10,
 10,
 10,
 10,
 1,
 10,
 10,
 14,
 3,
 18,
 18,
 6,
 10,
 1,
 14,
 18,
 18,
 10,
 10,
 10,
 10,
 11,
 10,
 10,
 10,
 11,
 1,
 10,
 10,
 1,
 10,
 10,
 4,
 10,
 3,
 18,
 18,
 11,
 10,
 2,
 2,
 10,
 10,
 10,
 1,
 10,
 18,
 11,
 7,
 10,
 18,
 18,
 10,
 10,
 10,
 10,
 15,
 10,
 1,
 18,
 17,
 18,
 4,
 1,
 10,
 15,
 11,
 10,
 3,
 10,
 11,
 11,
 1,
 18,
 10,
 10,
 1,
 10,
 3,
 10,
 11,
 2,
 10,
 15,
 17,
 17,
 10,
 13,
 18,
 18,
 15,
 18,
 10,
 14,
 10,
 10,
 18,
 2,
 10,
 10,
 3,
 10,
 18,
 10,
 10,
 6,
 10,
 10,
 10,
 16,
 11,
 10,
 10,
 10,
 10,
 10,
 11,
 10,
 10,
 1,
 10,
 2,
 10,
 10,
 18,
 10,
 10,
 1,
 11,
 10,
 10,
 1,
 10,
 10,
 11,
 18,
 3,
 18,
 1,
 10,
 10,
 18,
 13,
 10,
 10,
 1,
 4,
 17,
 2,
 10,
 10,
 10,
 18,
 10,
 10,
 10,
 10,
 6,
 10,
 18,
 3,
 10,
 6,
 10,
 17,
 11,
 3,
 10,
 2,
 18,
 10,
 10,
 18,
 3,
 10,
 3,
 18,
 3,
 10,
 10,
 14,
 18,
 2,
 18,
 13,
 10,
 1,
 10,
 14,
 10,
 15,
 5,
 10,
 7,
 1,
 10,
 10,


#### 제출

In [24]:
submit = pd.read_csv('C:/Users/pc/OneDrive/바탕 화면/4-1/도배_하자/open/sample_submission.csv')
submit

,id,label
0,TEST_000,훼손
1,TEST_001,훼손
2,TEST_002,훼손
3,TEST_003,훼손
4,TEST_004,훼손
...,...,...
787,TEST_787,훼손
788,TEST_788,훼손
789,TEST_789,훼손
790,TEST_790,훼손


In [28]:
submit['label'] = result

In [29]:
submit

,id,label
0,TEST_000,10
1,TEST_001,10
2,TEST_002,3
3,TEST_003,6
4,TEST_004,10
...,...,...
787,TEST_787,3
788,TEST_788,12
789,TEST_789,12
790,TEST_790,10


In [35]:
labels = submit['label']
converted_labels = []  # 변환된 라벨을 저장할 리스트

for label in labels:
    if label == 0:
        converted_labels.append('가구수정')
    elif label == 1:
        converted_labels.append('걸레받이수정')
    elif label == 2:
        converted_labels.append('곰팡이')
    elif label == 3:
        converted_labels.append('꼬임')
    elif label == 4:
        converted_labels.append('녹오염')
    elif label == 5:
        converted_labels.append('들뜸') 
    elif label == 6:
        converted_labels.append('면불량') 
    elif label == 7:
        converted_labels.append('몰딩수정') 
    elif label == 8:
        converted_labels.append('반점') 
    elif label == 9:
        converted_labels.append('석고수정') 
    elif label == 10:
        converted_labels.append('오염') 
    elif label == 11:
        converted_labels.append('오타공') 
    elif label == 12:
        converted_labels.append('울음')  
    elif label == 13:
        converted_labels.append('이음부불량')
    elif label == 14:
        converted_labels.append('창틀,문틀수정')
    elif label == 15:
        converted_labels.append('터짐')
    elif label == 16:
        converted_labels.append('틈새과다')
    elif label == 17:
        converted_labels.append('피스')  
    elif label == 18:
        converted_labels.append('훼손') 

print(converted_labels)  # 변환된 라벨 리스트 출력


['오염', '오염', '꼬임', '면불량', '오염', '피스', '오염', '오염', '꼬임', '오타공', '걸레받이수정', '면불량', '오염', '오염', '오염', '오염', '오염', '걸레받이수정', '오염', '오염', '창틀,문틀수정', '꼬임', '훼손', '훼손', '면불량', '오염', '걸레받이수정', '창틀,문틀수정', '훼손', '훼손', '오염', '오염', '오염', '오염', '오타공', '오염', '오염', '오염', '오타공', '걸레받이수정', '오염', '오염', '걸레받이수정', '오염', '오염', '녹오염', '오염', '꼬임', '훼손', '훼손', '오타공', '오염', '곰팡이', '곰팡이', '오염', '오염', '오염', '걸레받이수정', '오염', '훼손', '오타공', '몰딩수정', '오염', '훼손', '훼손', '오염', '오염', '오염', '오염', '터짐', '오염', '걸레받이수정', '훼손', '피스', '훼손', '녹오염', '걸레받이수정', '오염', '터짐', '오타공', '오염', '꼬임', '오염', '오타공', '오타공', '걸레받이수정', '훼손', '오염', '오염', '걸레받이수정', '오염', '꼬임', '오염', '오타공', '곰팡이', '오염', '터짐', '피스', '피스', '오염', '이음부불량', '훼손', '훼손', '터짐', '훼손', '오염', '창틀,문틀수정', '오염', '오염', '훼손', '곰팡이', '오염', '오염', '꼬임', '오염', '훼손', '오염', '오염', '면불량', '오염', '오염', '오염', '틈새과다', '오타공', '오염', '오염', '오염', '오염', '오염', '오타공', '오염', '오염', '걸레받이수정', '오염', '곰팡이', '오염', '오염', '훼손', '오염', '오염', '걸레받이수정', '오타공', '오염', '오염', '걸레받이수정', '오염', '오염', '오타공', '훼손', '꼬임', '

In [37]:
submit['label'] = converted_labels
submit

,id,label
0,TEST_000,오염
1,TEST_001,오염
2,TEST_002,꼬임
3,TEST_003,면불량
4,TEST_004,오염
...,...,...
787,TEST_787,꼬임
788,TEST_788,울음
789,TEST_789,울음
790,TEST_790,오염


In [39]:
submit['label'].value_counts()

오염         358
훼손          87
꼬임          76
오타공         52
걸레받이수정      43
곰팡이         37
면불량         37
터짐          24
창틀,문틀수정     14
피스          14
녹오염         12
몰딩수정         9
들뜸           9
이음부불량        7
가구수정         6
석고수정         3
울음           3
틈새과다         1
Name: label, dtype: int64

In [38]:
submit.to_csv('./cnn2_submit.csv', index=False)

### f1 score

In [ ]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test, pred)
print('F1 스코어 : {0:.4f}'.format(f1))

In [ ]:
from sklearn.metrics import f1_score

# 예측값과 실제값을 받아 F1 점수를 계산하는 함수
def calculate_f1_score(predictions, labels):
    f1 = f1_score(labels, predictions)
    return f1

# 예시 데이터
predictions = [1, 0, 1, 1, 0]
labels = [1, 1, 1, 0, 1]

# F1 점수 계산
f1_score = calculate_f1_score(predictions, labels)
print("F1 점수:", f1_score)


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# 정밀도 계산
precision = precision_score(train_gen, test)
print("Precision:", precision)

# 재현율 계산
recall = recall_score(train_gen, test)
print("Recall:", recall)

# F1 스코어 계산
f1 = f1_score(train_gen, test)
print("F1 Score:", f1)
